In [1]:
import pandas as pd
import requests
import tqdm
import time
import bs4
import json
from datetime import datetime

In [2]:
# ambil web bbc
news_request = requests.get("https://www.bbc.com/news")
news_request

# ubah content menjadi objek bs4
news_bs4 = bs4.BeautifulSoup(news_request.content)
# print(news_bs4.prettify())

# card article 
news_bs4_cards_1 = news_bs4.find(name='script', attrs={'id': '__NEXT_DATA__'})
path_news = json.loads(news_bs4_cards_1.text)['props']['pageProps']['page']['@"news",']['sections']

In [3]:
list_news = []
for news in tqdm.tqdm(path_news) : 
    if 'content' in news.keys() : 
        for i in news['content'] : 
            content = {
                'title'                : i['title']
                , 'publication_date'   : datetime.fromtimestamp(i['metadata']['firstUpdated']/1000)
                , 'category'           : i['metadata']['contentType']
                , 'subcategory'        : i['metadata']['subtype'] 
                , 'url'                : "https://bbc.com"+i['href'] if 'bbc.com' not in i['href'] else i['href']
                , 'summary'            : i['description']
            }
            list_news.append(content)

            time.sleep(0.1)

len(list_news)

100%|██████████| 10/10 [00:04<00:00,  2.03it/s]


49

In [4]:
df_raw = pd.DataFrame(list_news)
df_raw.head()

,title,publication_date,category,subcategory,url,summary
0,Europe heatwave puts Paris on red alert as par...,2025-06-30 12:41:00.000,live,news,https://www.bbc.com/news/live/cd0v29vydx4t,Two people have died in Italy and tens of thou...
1,US Senate holds marathon overnight vote on Tru...,2025-06-30 21:09:37.583,article,news,https://bbc.com/news/articles/czjkkdyplymo,"Senators are debating a nearly 1,000-page bill..."
2,Thai prime minister suspended over leaked phon...,2025-07-01 13:44:07.968,article,news,https://bbc.com/news/articles/c9vrrj72xy7o,Paetongtarn Shinawatra has been suspended as t...
3,US-Israeli backed Gaza aid group must be shut ...,2025-07-01 12:02:06.652,article,news,https://bbc.com/news/articles/cn5kk1w00xyo,The call from some of the world's top aid orga...
4,Is the India-US trade deal in trouble?,2025-07-01 13:40:48.800,article,news,https://bbc.com/news/articles/c625504qp3go,"With days to go before 9 July, when the tariff..."


In [5]:
## drop duplicates

df = df_raw.drop_duplicates(subset=['url'], keep='last')

In [6]:
## check null values
df.isnull().sum()

title               0
publication_date    0
category            0
subcategory         0
url                 0
summary             0
dtype: int64

In [7]:
## save to csv
df.to_csv('output_bbc_scrap.csv', encoding='utf-8', index=False)